Churn

In [1]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt

In [29]:

# Read data
df = pd.read_json('data/mini_sparkify_event_data.json', orient='records', lines=True)
df.head(5)

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771


In [72]:


pd.to_datetime(
    datetime.utcfromtimestamp(df.iloc[0]['registration']/1000)
) == \
pd.Timestamp(datetime.utcfromtimestamp(df.iloc[0]['registration']/1000))

True

In [30]:


def convert_ts(x):
    dt = datetime.utcfromtimestamp(int(x)/1000)
    return dt

In [24]:


datetime.utcfromtimestamp(df.iloc[0]['registration']/1000) == pd.Timestamp(datetime.utcfromtimestamp(df.iloc[0]['registration']/1000))

True

In [31]:


df['datetime'] = df.ts.apply(convert_ts)
df

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length,datetime
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016,2018-10-01 00:01:57
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424,2018-10-01 00:03:00
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730,2018-10-01 00:06:34
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302,2018-10-01 00:06:56
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771,2018-10-01 00:11:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286495,1543622240000,,500,Home,Logged Out,GET,200,paid,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-30 23:57:20
286496,1543622241000,,500,Login,Logged Out,PUT,307,paid,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-30 23:57:21
286497,1543622248000,300011,500,Home,Logged In,GET,200,paid,43,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,House,Emilia,1.538337e+12,F,NaN,NaN,NaN,2018-11-30 23:57:28
286498,1543622398000,300011,500,About,Logged In,GET,200,paid,44,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,House,Emilia,1.538337e+12,F,NaN,NaN,NaN,2018-11-30 23:59:58


In [71]:


# drop anonymous

df = df[df['userId']!='']

# convert userid type

df['userId'] = df['userId'].astype('int')

0             30
1              9
2             30
3              9
4             30
           ...  
286492    300011
286493    300011
286497    300011
286498    300011
286499    300011
Name: userId, Length: 278154, dtype: int64

In [73]:
df

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length,datetime
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016,2018-10-01 00:01:57
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424,2018-10-01 00:03:00
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730,2018-10-01 00:06:34
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302,2018-10-01 00:06:56
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771,2018-10-01 00:11:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286492,1543622121000,300011,500,NextSong,Logged In,PUT,200,paid,38,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,House,Emilia,1.538337e+12,F,Iron Maiden,Murders In The Rue Morgue (1998 Digital Remaster),258.66404,2018-11-30 23:55:21
286493,1543622122000,300011,500,Logout,Logged In,PUT,307,paid,39,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,House,Emilia,1.538337e+12,F,NaN,NaN,NaN,2018-11-30 23:55:22
286497,1543622248000,300011,500,Home,Logged In,GET,200,paid,43,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,House,Emilia,1.538337e+12,F,NaN,NaN,NaN,2018-11-30 23:57:28
286498,1543622398000,300011,500,About,Logged In,GET,200,paid,44,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,House,Emilia,1.538337e+12,F,NaN,NaN,NaN,2018-11-30 23:59:58


In [75]:
df['reg'] = df['registration'].apply(convert_ts)

df['reg']

0        2018-09-28 22:22:42
1        2018-09-30 18:20:30
2        2018-09-28 22:22:42
3        2018-09-30 18:20:30
4        2018-09-28 22:22:42
                 ...        
286492   2018-09-30 19:46:11
286493   2018-09-30 19:46:11
286497   2018-09-30 19:46:11
286498   2018-09-30 19:46:11
286499   2018-09-30 19:46:11
Name: reg, Length: 278154, dtype: datetime64[ns]

In [78]:
df.dtypes

ts                        int64
userId                    int64
sessionId                 int64
page                     object
auth                     object
method                   object
status                    int64
level                    object
itemInSession             int64
location                 object
userAgent                object
lastName                 object
firstName                object
registration            float64
gender                   object
artist                   object
song                     object
length                  float64
datetime         datetime64[ns]
reg              datetime64[ns]
dtype: object

In [80]:
df['userAgent'].value_counts()

"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                                22751
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"                        19611
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"                     18448
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                     17348
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0                                                                       16700
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"                                15395
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/53

In [84]:
df['gender'].value_counts() / df['gender'].value_counts().sum()

F    0.555728
M    0.444272
Name: gender, dtype: float64

In [87]:
df['status'].value_counts() / df['status'].value_counts().sum()

200    0.915745
307    0.083350
404    0.000906
Name: status, dtype: float64

In [155]:
n_users = df['userId'].nunique()
n_users

225

In [110]:
df.groupby(['userId']).size()

userId
2          899
3          254
4         2442
5          218
6         3761
          ... 
300021    4659
300022     446
300023    2210
300024     102
300025    1603
Length: 225, dtype: int64

In [114]:
df[df['level'] == 'paid']['userId'].nunique()

165

In [115]:
df[df['level'] == 'free']['userId'].nunique()

195

In [146]:
subscriber_user_ids = df[df['level'] == 'paid']['userId'].value_counts().index.to_list()
subscriber_user_ids

[39,
 92,
 140,
 300011,
 124,
 300021,
 300017,
 42,
 85,
 6,
 29,
 300015,
 9,
 100,
 91,
 54,
 126,
 118,
 74,
 51,
 98,
 136,
 65,
 4,
 81,
 15,
 132,
 142,
 41,
 138,
 88,
 300023,
 200004,
 127,
 101,
 300001,
 300018,
 128,
 200008,
 115,
 96,
 97,
 60,
 58,
 300002,
 62,
 20,
 25,
 120,
 113,
 104,
 131,
 200023,
 300009,
 300008,
 70,
 53,
 37,
 45,
 33,
 300025,
 82,
 38,
 14,
 152,
 114,
 145,
 69,
 95,
 61,
 200021,
 28,
 83,
 77,
 153,
 67,
 100013,
 103,
 17,
 52,
 35,
 46,
 300019,
 75,
 40,
 141,
 49,
 2,
 105,
 106,
 100008,
 36,
 155,
 66,
 10,
 109,
 300012,
 300016,
 111,
 100018,
 89,
 59,
 30,
 16,
 12,
 86,
 23,
 21,
 121,
 108,
 100004,
 200009,
 100016,
 56,
 100007,
 18,
 44,
 99,
 147,
 100023,
 13,
 300022,
 73,
 200025,
 200019,
 139,
 100025,
 300013,
 200017,
 100015,
 129,
 200011,
 200002,
 300014,
 200020,
 300005,
 100009,
 100014,
 300020,
 19,
 100012,
 3,
 93,
 71,
 11,
 100002,
 200024,
 200014,
 151,
 50,
 200003,
 300006,
 87,
 300004,
 300007,


In [137]:
free_user_ids = df[~df['userId'].isin(subscriber_user_ids)]['userId'].value_counts().index.to_list()
free_user_ids

[100022,
 146,
 200006,
 148,
 200018,
 80,
 200022,
 117,
 100010,
 300010,
 200015,
 8,
 78,
 100021,
 200010,
 107,
 112,
 27,
 76,
 200016,
 47,
 43,
 149,
 110,
 150,
 119,
 5,
 100005,
 102,
 7,
 123,
 94,
 100001,
 200001,
 143,
 144,
 154,
 72,
 57,
 63,
 84,
 100019,
 200012,
 116,
 100003,
 100017,
 34,
 64,
 134,
 68,
 90,
 100006,
 133,
 22,
 100024,
 300003,
 100011,
 125,
 156,
 135]

In [147]:
len(free_user_ids) + len(subscriber_user_ids)

225

In [142]:
df['auth'].value_counts()

Logged In    278102
Cancelled        52
Name: auth, dtype: int64

In [153]:
churn_user_ids = df[df['auth']=='Cancelled']['userId'].value_counts().index.to_list()
churn_user_ids

[18,
 32,
 100003,
 100023,
 100005,
 100017,
 100025,
 100009,
 100012,
 100013,
 100022,
 100014,
 100015,
 100021,
 100007,
 200001,
 200024,
 200011,
 200021,
 200018,
 200015,
 200017,
 200016,
 200020,
 300007,
 100019,
 100006,
 100024,
 12,
 125,
 105,
 17,
 143,
 101,
 129,
 121,
 51,
 87,
 122,
 58,
 100001,
 73,
 3,
 106,
 103,
 28,
 54,
 29,
 70,
 53,
 100011,
 300001]

In [163]:
df[df['auth']=='Cancelled'].groupby('level').size()

level
free    21
paid    31
dtype: int64

In [157]:
# overall churn.
# total number of churn users / total number of users

len(churn_user_ids) / n_users

0.2311111111111111

In [164]:
# free users churn
# total number of churn users who are 'free-tier' / total number of free users

df[df['auth']=='Cancelled'].groupby('level').size()['free'] / len(free_user_ids)

0.35

In [165]:
# paid users churn
# total number of churn users who are subscriber 'paid-tier' / total number of subscriber users

df[df['auth']=='Cancelled'].groupby('level').size()['paid'] / len(subscriber_user_ids)

0.18787878787878787